# Prophet Baseline Notebook (Lean Version)

Purpose: Establish a transparent statistical baseline for 30-day item-level demand forecasting on the M5 subset.
Scope: CPU-only, small item subset (<=10) + aggregate series; hold-out last 30 days for validation.
Artifacts: metrics JSON saved to artifacts/models/prophet_metrics.json for README results table.

Steps:
1. Load processed panel (or synthetic fallback identical to data_prep).
2. Train/validation split (last FORECAST_LENGTH days).
3. Fit Prophet on aggregate series -> forecast -> compute WAPE/MAE.
4. Fit Prophet per item (first N_ITEMS) -> hold-out validation metrics.
5. Save aggregated metrics JSON.

Metric definitions: WAPE (weighted abs pct error), Accuracy = 100 - WAPE.

Run order: Execute all cells top-to-bottom; then fill README results placeholders.

In [19]:
# Path injection to ensure 'src' package resolvable when notebook launched from notebooks/
import sys
from pathlib import Path

_nb_root = Path.cwd()
# If current working dir ends with 'notebooks', ascend one level for repo root
_repo_root = _nb_root if (_nb_root / 'src').exists() else _nb_root.parent
_src = _repo_root / 'src'

if _src.exists():
    paths_added = []
    for p in (str(_repo_root), str(_src)):
        if p not in sys.path:
            sys.path.insert(0, p)
            paths_added.append(p)
    print('sys.path augmented with:', paths_added)
else:
    print('Warning: src directory not found at expected path:', _src)
    print('CWD:', _nb_root)
    print('Directory listing of parent:', [d for d in _nb_root.parent.iterdir()][:15])

try:
    import src
    print('Import test: src package available.')
except ModuleNotFoundError:
    print('ModuleNotFoundError persists. Head of sys.path:', sys.path[:8])
    # Provide troubleshooting hints
    print('Troubleshooting: Ensure you started Jupyter from repo root or run "cd .." before opening notebook.')
    raise

sys.path augmented with: []
Import test: src package available.


In [20]:
# Imports & constants
import pandas as pd, numpy as np, json, math
from pathlib import Path
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
try:
    from prophet import Prophet
except ImportError:
    print('Prophet not installed. Install with `pip install prophet` and rerun.')
    Prophet = None
from src.evaluation.metrics import wape, mae
PANEL_PATH = Path('data/processed/m5_panel_subset.parquet')
ARTIFACT_DIR = Path('artifacts/models')
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)
FORECAST_LENGTH = 30
INPUT_LENGTH = 28 * 4  # aligns with N-BEATS history
N_ITEMS = 10  # limit for lean baseline
SEED = 42
np.random.seed(SEED)


In [ ]:
# Load panel or synthetic fallback
if PANEL_PATH.exists():
    panel_df = pd.read_parquet(PANEL_PATH)
    source_note = f'Loaded panel: {PANEL_PATH}'
else:
    print('Panel parquet missing -> generating synthetic fallback (20 items x 200 days).')
    items = [f'ITEM_{i:03d}' for i in range(20)]
    dates = pd.date_range('2024-01-01', periods=200, freq='D')
    rows = []
    for item in items:
        base = np.random.randint(5, 25)
        seasonal = np.sin(np.linspace(0, 12 * math.pi, len(dates))) * np.random.uniform(3, 8)
        noise = np.random.randn(len(dates)) * np.random.uniform(0.5, 2.0)
        demand = (base + seasonal + noise).clip(min=0).round(2)
        for d, val in zip(dates, demand):
            rows.append({'item_id': item, 'date': d, 'demand': float(val)})
    panel_df = pd.DataFrame(rows)
    # Ensure directory exists before saving
    PANEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    panel_df.to_parquet(PANEL_PATH, index=False)
    source_note = 'Synthetic fallback saved.'
panel_df['date'] = pd.to_datetime(panel_df['date'])
print(source_note)
print(panel_df.head())


Panel parquet missing -> generating synthetic fallback (20 items x 200 days).


OSError: Cannot save file into a non-existent directory: 'data/processed'

In [ ]:
# Train/validation split (hold out last FORECAST_LENGTH days)
unique_dates = sorted(panel_df['date'].unique())
assert len(unique_dates) > FORECAST_LENGTH, 'Not enough days for hold-out validation.'
cutoff_date = unique_dates[-FORECAST_LENGTH]  # first day of validation segment
train_df = panel_df[panel_df['date'] < cutoff_date]
valid_df = panel_df[panel_df['date'] >= cutoff_date]
print('Cutoff date:', cutoff_date)
print('Train range:', train_df['date'].min().date(), '->', train_df['date'].max().date(), '| rows:', len(train_df))
print('Valid range:', valid_df['date'].min().date(), '->', valid_df['date'].max().date(), '| rows:', len(valid_df))


In [ ]:
# Aggregate series modeling with Prophet
if Prophet is None:
    print('Skip aggregate model: Prophet not installed.')
else:
    agg_train = train_df.groupby('date', as_index=False)['demand'].sum().rename(columns={'date':'ds','demand':'y'})
    agg_valid = valid_df.groupby('date', as_index=False)['demand'].sum().rename(columns={'date':'ds','demand':'y'})
    model_agg = Prophet()
    model_agg.fit(agg_train)
    future_agg = model_agg.make_future_dataframe(periods=FORECAST_LENGTH)
    fc_agg = model_agg.predict(future_agg)
    fc_valid = fc_agg.tail(FORECAST_LENGTH)
    # Align with actual validation dates
    actual_valid = agg_valid['y'].values
    pred_valid = fc_valid['yhat'].values
    agg_wape = wape(actual_valid, pred_valid)
    agg_mae = mae(actual_valid, pred_valid)
    agg_accuracy = 100 - agg_wape if not math.isnan(agg_wape) else float('nan')
    print(f'Agg validation WAPE: {agg_wape:.2f} | MAE: {agg_mae:.2f} | Accuracy: {agg_accuracy:.2f}')

In [ ]:
# Per-item modeling loop (subset)
item_ids = sorted(train_df['item_id'].unique())[:N_ITEMS]
item_metric_rows = []
if Prophet is None:
    print('Skip per-item loop: Prophet not installed.')
else:
    for itm in item_ids:
        sub_train = train_df[train_df.item_id == itm].sort_values('date')
        sub_valid = valid_df[valid_df.item_id == itm].sort_values('date')
        # Require minimum history
        if len(sub_train) < INPUT_LENGTH:
            continue
        df_train = sub_train[['date','demand']].rename(columns={'date':'ds','demand':'y'})
        df_valid = sub_valid[['date','demand']].rename(columns={'date':'ds','demand':'y'})
        m_item = Prophet()
        m_item.fit(df_train)
        future_item = m_item.make_future_dataframe(periods=FORECAST_LENGTH)
        fc_item = m_item.predict(future_item).tail(FORECAST_LENGTH)
        y_true = df_valid['y'].values
        y_pred = fc_item['yhat'].values
        w = wape(y_true, y_pred)
        a = mae(y_true, y_pred)
        item_metric_rows.append({'item_id': itm, 'wape': w, 'mae': a})
    per_item_df = pd.DataFrame(item_metric_rows)
    if not per_item_df.empty:
        mean_wape = per_item_df['wape'].mean()
        mean_mae = per_item_df['mae'].mean()
        print(f'Per-item mean WAPE (subset): {mean_wape:.2f} | MAE: {mean_mae:.2f}')
    else:
        mean_wape = float('nan'); mean_mae = float('nan')

In [ ]:
# Save metrics artifact
metrics = {
    'source': source_note,
    'forecast_length': FORECAST_LENGTH,
    'input_length': INPUT_LENGTH,
    'aggregate_wape': agg_wape if 'agg_wape' in globals() else None,
    'aggregate_mae': agg_mae if 'agg_mae' in globals() else None,
    'aggregate_accuracy': agg_accuracy if 'agg_accuracy' in globals() else None,
    'per_item_mean_wape': mean_wape if 'mean_wape' in globals() else None,
    'per_item_mean_mae': mean_mae if 'mean_mae' in globals() else None,
    'n_items_modeled': len(item_metric_rows) if 'item_metric_rows' in globals() else 0,
    'seed': SEED,
}
out_path = ARTIFACT_DIR / 'prophet_metrics.json'
with open(out_path,'w') as f: json.dump(metrics, f, indent=2)
print('Saved Prophet metrics ->', out_path)
metrics

In [ ]:
# Notebook summary markdown (for human-readable insight)
from pathlib import Path
md_lines = [
    '# Prophet Baseline Summary',
    f'Aggregate WAPE: {metrics.get("aggregate_wape")}',
    f'Aggregate MAE: {metrics.get("aggregate_mae")}',
    f'Per-item Mean WAPE: {metrics.get("per_item_mean_wape")}',
    f'Items Modeled: {metrics.get("n_items_modeled")}',
    f'Forecast Length: {FORECAST_LENGTH}',
    '',
    'Use these values to populate the README results table.'
]
summary_path = ARTIFACT_DIR / 'prophet_metrics_summary.md'
with open(summary_path,'w') as f: f.write('\n'.join(md_lines))
print('Wrote summary markdown ->', summary_path)

In [ ]:
# Next steps note (markdown)
print('Next: Run nbeats_training.ipynb to compare metrics and generate model card.')